# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"> **Air Quality** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Batch Inference</span>

## 🗒️ This notebook is divided into the following sections:

1. Download model and batch inference data
2. Make predictions, generate PNG for forecast
3. Store predictions in a monitoring feature group adn generate PNG for hindcast

## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Read the API keys and configuration variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

In [ ]:
import datetime
import pandas as pd
from xgboost import XGBRegressor
import hopsworks
import json
from mlfs.airquality import util
import os

In [ ]:
today = datetime.datetime.now() - datetime.timedelta(0)
tomorrow = today + datetime.timedelta(days = 1)
today

## <span style="color:#ff5f27;"> 📡 Connect to Hopsworks Feature Store </span>

In [ ]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 

secrets = hopsworks.get_secrets_api()
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)
country=location['country']
city=location['city']
street=location['street']

## <span style="color:#ff5f27;">🪝 Download the model from Model Registry</span>

In [ ]:
mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="air_quality_xgboost_model",
    version=1,
)

fv = retrieved_model.get_feature_view()

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

In [ ]:
# Loading the XGBoost regressor model and label encoder from the saved model directory
# retrieved_xgboost_model = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
retrieved_xgboost_model = XGBRegressor()

retrieved_xgboost_model.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
retrieved_xgboost_model

## <span style="color:#ff5f27;">✨ Get Weather Forecast Features with Feature View   </span>



In [ ]:
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)
batch_data = weather_fg.filter(weather_fg.date >= today).read()
batch_data

In [ ]:
# Get recent PM2.5 history to compute lag features for predictions (Grade C requirement)
air_quality_fg = fs.get_feature_group(name='air_quality', version=1)

# Ensure dates are in datetime format for proper comparison
batch_data['date'] = pd.to_datetime(batch_data['date'])

# Read all historical PM2.5 data for this street (we need enough history for lag features)
historical_pm25 = (
    air_quality_fg.read()
    .query("street == @street")
    .sort_values('date')
    [['date', 'pm25']]
)
historical_pm25['date'] = pd.to_datetime(historical_pm25['date'])

# For each prediction date, compute lag features from historical data
# We need to find the PM2.5 values 1, 2, and 3 days before each prediction date
for idx, pred_date in enumerate(batch_data['date']):
    # Find PM2.5 values for 1, 2, and 3 days before the prediction date
    lag_1_date = pred_date - datetime.timedelta(days=1)
    lag_2_date = pred_date - datetime.timedelta(days=2)
    lag_3_date = pred_date - datetime.timedelta(days=3)
    
    # Get PM2.5 values for those dates
    lag_1_val = historical_pm25[historical_pm25['date'] == lag_1_date]['pm25'].values
    lag_2_val = historical_pm25[historical_pm25['date'] == lag_2_date]['pm25'].values
    lag_3_val = historical_pm25[historical_pm25['date'] == lag_3_date]['pm25'].values
    
    # Assign lag features (use 0.0 if date not found)
    batch_data.at[batch_data.index[idx], 'pm25_lag_1'] = lag_1_val[0] if len(lag_1_val) > 0 else 0.0
    batch_data.at[batch_data.index[idx], 'pm25_lag_2'] = lag_2_val[0] if len(lag_2_val) > 0 else 0.0
    batch_data.at[batch_data.index[idx], 'pm25_lag_3'] = lag_3_val[0] if len(lag_3_val) > 0 else 0.0
    
    # Compute rolling 3-day mean using the 3 lag values
    lag_values = [batch_data.at[batch_data.index[idx], 'pm25_lag_1'],
                  batch_data.at[batch_data.index[idx], 'pm25_lag_2'],
                  batch_data.at[batch_data.index[idx], 'pm25_lag_3']]
    # Count non-zero values for proper mean calculation
    non_zero_values = [v for v in lag_values if v != 0.0]
    if len(non_zero_values) > 0:
        batch_data.at[batch_data.index[idx], 'pm25_roll_3'] = sum(non_zero_values) / len(non_zero_values)
    else:
        batch_data.at[batch_data.index[idx], 'pm25_roll_3'] = 0.0

# Ensure lag features are float type
batch_data[['pm25_lag_1', 'pm25_lag_2', 'pm25_lag_3', 'pm25_roll_3']] = \
    batch_data[['pm25_lag_1', 'pm25_lag_2', 'pm25_lag_3', 'pm25_roll_3']].astype('float32')

print("Lag features added to batch_data:")
print(batch_data[['date', 'pm25_lag_1', 'pm25_lag_2', 'pm25_lag_3', 'pm25_roll_3']].head())
batch_data


### <span style="color:#ff5f27;">🤖 Making the predictions</span>

In [ ]:
# old --> batch_data['predicted_pm25'] = retrieved_xgboost_model.predict(
    #old --> batch_data[['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant']])

# added features for the new features (c level)
# Include lag features in prediction (model expects lag features first, then weather features - Grade C requirement)
feature_cols = [
    'pm25_lag_1',
    'pm25_lag_2',
    'pm25_lag_3',
    'pm25_roll_3',
    'temperature_2m_mean',
    'precipitation_sum',
    'wind_speed_10m_max',
    'wind_direction_10m_dominant'
]

batch_data['predicted_pm25'] = retrieved_xgboost_model.predict(batch_data[feature_cols])
batch_data

In [ ]:
batch_data.info()

### <span style="color:#ff5f27;">🤖 Saving the predictions (for monitoring) to a Feature Group</span>

In [ ]:
batch_data['street'] = street
batch_data['city'] = city
batch_data['country'] = country
# Fill in the number of days before the date on which you made the forecast (base_date)
batch_data['days_before_forecast_day'] = range(1, len(batch_data)+1)
batch_data = batch_data.sort_values(by=['date'])
batch_data

In [ ]:
batch_data.info()

### Create Forecast Graph
Draw a graph of the predictions with dates as a PNG and save it to the github repo
Show it on github pages

In [ ]:

pred_file_path = f"{root_dir}/docs/air-quality/assets/img/pm25_forecast.png"
plt = util.plot_air_quality_forecast(city, street, batch_data, pred_file_path)

plt.show()

In [ ]:
# Get or create feature group
monitor_fg = fs.get_or_create_feature_group(
    name='aq_predictions',
    description='Air Quality prediction monitoring',
    version=1,
    primary_key=['city','street','date','days_before_forecast_day'],
    event_time="date"
)

In [ ]:
monitor_fg.insert(batch_data, wait=True)

In [ ]:
# We will create a hindcast chart for  only the forecasts made 1 day beforehand
monitoring_df = monitor_fg.filter(monitor_fg.days_before_forecast_day == 1).read()
monitoring_df

In [ ]:
air_quality_fg = fs.get_feature_group(name='air_quality', version=1)
air_quality_df = air_quality_fg.read()
air_quality_df

In [ ]:
outcome_df = air_quality_df[['date', 'pm25']]
preds_df =  monitoring_df[['date', 'predicted_pm25']]

hindcast_df = pd.merge(preds_df, outcome_df, on="date")
hindcast_df = hindcast_df.sort_values(by=['date'])

# If there are no outcomes for predictions yet, generate some predictions/outcomes from existing data
if len(hindcast_df) == 0:
    hindcast_df = util.backfill_predictions_for_monitoring(weather_fg, air_quality_df, monitor_fg, retrieved_xgboost_model)
hindcast_df

### Plot the Hindcast comparing predicted with forecasted values (1-day prior forecast)

__This graph will be empty to begin with - this is normal.__

After a few days of predictions and observations, you will get data points in this graph.

In [ ]:
hindcast_file_path = f"{root_dir}/docs/air-quality/assets/img/pm25_hindcast_1day.png"
plt = util.plot_air_quality_forecast(city, street, hindcast_df, hindcast_file_path, hindcast=True)
plt.show()

### Upload the prediction and hindcast dashboards (png files) to Hopsworks


In [ ]:
dataset_api = project.get_dataset_api()
str_today = today.strftime("%Y-%m-%d")
if dataset_api.exists("Resources/airquality") == False:
    dataset_api.mkdir("Resources/airquality")
dataset_api.upload(pred_file_path, f"Resources/airquality/{city}_{street}_{str_today}", overwrite=True)
dataset_api.upload(hindcast_file_path, f"Resources/airquality/{city}_{street}_{str_today}", overwrite=True)

proj_url = project.get_url()
print(f"See images in Hopsworks here: {proj_url}/settings/fb/path/Resources/airquality")

---